# Table of Contents
1. [Introduction: ETL with Python and MySQL](#p1)
2. [Importing Libraries](#p2)
3. [Connect to Database](#p3)
4. [Data Preparation](#p4)
5. [Data Visualization](#p5)
6. [Visualization with Plotly](#p6)

## <a name="p1">Introduction: ETL with Python and MySQL</a>

Data is the gold for the modern business world. Although, Data on its own is not very useful. Furthermore, application databases store data in a complex way to use for analytics.
 
ETL tools are essential because they take data from multiple systems and combine them into a single database, data warehouse or data store for storage or analytics. These tools and services allow enterprises to set up a data pipeline and begin ingesting data quickly. `Analysts` and `BI Analysts` sort, join, reformat, filter, merge and aggregate data to make it more meaningful for business decisions. Subsequently, they include graphical interfaces for faster, more accessible results than traditional moving data methods through hand-coded data pipelines.
 
Analysts and engineers can use Python to build their ETL pipelines; as it has dominated the industry for a few years now, hundreds of Python tools act as software, libraries, or framework for ETL. Moreover, Python allows them to control and customize each aspect of the pipeline, but a handmade one also requires more time and effort to create and maintain. 
 
I decided to work with `Pandas`, a Python library that provides you with data structures and analysis tools.; you can install it by typing the following command in your command prompt or Jupyter Notebook cell.
 
- Command Prompt: pip install pandas
- Jupyter Notebook: !pip install pandas
 
Pandas simplify all processes like data cleansing by adding R-style data frames. Therefore, it is time-taking to use as you would have to write your code. But, it can write simple scripts quickly.
So, when it comes to scalability and in-memory, Pandas’ performance may not keep up with expectations. It would help if you used pandas when you need to extract data, clean, transform it rapidly, and write it to an SQL database/Excel/csv. 
 
However, when you start working with large data sets, it usually makes more sense to use a more scalable approach.
 
After you installed Pandas, you will need a local server and `MySQL` driver to extract data from our environment. My preference is `XAMPP` you can easily download, and to get the same driver as it is in my tutorial, either on your Command Prompt or Jupyter Notebook cell:
 
 
 - Command Prompt: pip install pymysql
 - Jupyter Notebook: !pip install pymysql 
 
There are other options and drivers to query a database, but I have a soft spot for MySQL; the setup will be slightly different, though they will work the same way, in case you have another choice. 
 
 
The SQL scripts will be in this same folder; I assume you can set it up yourselves, as I am not covering this topic. 


## <a name="p2">Import Libraries</a>

In [ ]:
import pandas as pd
import numpy as np
#!pip install pymysql
import pymysql

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import matplotlib.gridspec as gridspec

## <a name="p3">Connect to Database</a>

In [ ]:
connection = pymysql.connect(host='localhost', user='Lauro', password='Lauro12345', db='world')

city = pd.read_sql('select * from city', connection)
country = pd.read_sql('select * from country', connection)
countrylanguage = pd.read_sql('select * from country language', connection)

# Close the Connection
connection.close()

In [ ]:
city.head()

In [ ]:
country.head()

In [ ]:
countrylanguage.head()

## <a name="p4">Data Preparation</a>

In [ ]:
city = city.rename(columns={"Name": "City", "CountryCode": "Code", "Population":"City_Population"}, errors="raise")
country = country.rename(columns={"Name":"Country"})
countrylanguage = countrylanguage.rename(columns={"Name":"Country"})

In [ ]:
print(city.columns)
print("-------------------------------------------------------------------")
print(country.columns)
print("-------------------------------------------------------------------")
print(countrylanguage.columns)

In [ ]:
a = pd.merge(city, country)
df = pd.merge(a, countrylanguage)

df.head()

In [ ]:
df = df.drop(['Code', 'LocalName', 'HeadOfState','Code2','IndepYear', 'Capital'], axis=1)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df['LifeExpectancy'] = df['LifeExpectancy'].fillna(df['LifeExpectancy'].mean())

df['LifeExpectancy'] = df['LifeExpectancy'].astype(np.int64)

In [ ]:
df['GNPOld'] = df['GNPOld'].fillna(df['GNPOld'].mean())

In [ ]:
df.isna().any().any()

## <a name="p5">Data Visualization</a>

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import matplotlib.gridspec as gridspec

In [ ]:
x = df[['LifeExpectancy','GNP','Population','Continent']]

g=sns.pairplot(x, hue="Continent", diag_kind='hist')
g.fig.suptitle('Pairplot showing GNP per capita, Life Expectancy, Country against Continent',y=1.05)

In [ ]:
f,ax = plt.subplots(figsize = (15,10))
sns.heatmap(df.corr(),annot = True, linewidth = 0.5, fmt = ".1f", ax=ax)  #seaborn 
plt.show()

In [ ]:
#BAR GRAPH OF TOP 30 COUNTRIES WITH HIGHEST LIFE EXPECTANCY
mostExp30Data = country.sort_values("LifeExpectancy", ascending = False).head(30)

plt.figure(figsize = (10, 10))
sns.barplot(x = mostExp30Data["LifeExpectancy"], y = mostExp30Data["Country"])
plt.xticks(rotation = 90)
plt.title("Top 30 Countries with the Highest Life Expectancy")
plt.xlabel("Years")
plt.ylabel("Countries")
plt.show()

In [ ]:
#BAR GRAPH OF TOP 30 COUNTRIES WITH LOWEST LIFE EXPECTANCY
mostExp30Data = country.sort_values("LifeExpectancy", ascending = True).head(30)

plt.figure(figsize = (10, 10))
sns.barplot(x = mostExp30Data["LifeExpectancy"], y = mostExp30Data["Country"])
plt.xticks(rotation = 90)
plt.title("Top 30 Countries with the Lowest Life Expectancy")
plt.xlabel("Years")
plt.ylabel("Countries")
plt.show()

In [ ]:
#BAR GRAPH OF TOP 30 COUNTRIES WITH HIGHEST LIFE EXPECTANCY
mostExp30Data = df.sort_values("LifeExpectancy", ascending = False)

plt.figure(figsize = (10, 10))
sns.barplot(x = mostExp30Data["LifeExpectancy"], y = mostExp30Data["Continent"])
plt.xticks(rotation = 90)
plt.title("Top Continents with the Highest Life Expectancy")
plt.xlabel("Years")
plt.ylabel("Continents")
plt.show()

In [ ]:
#BAR GRAPH OF CONTINENTAL POPULATION
mostPopData = df.sort_values("GNP", ascending = False)

plt.figure(figsize = (10, 10))
sns.barplot(x = mostPopData["GNP"], y = mostPopData["Continent"])
plt.xticks(rotation = 90)
plt.title("GNP per Continent")
plt.xlabel("GNP")
plt.ylabel("Continent")
plt.show()

In [ ]:
#BAR GRAPH OF TOP 30 COUNTRIES WITH HIGHEST GNP
mostPop30Data = country.sort_values("GNP", ascending = False).head(30)

plt.figure(figsize = (10, 10))
sns.barplot(x = mostPop30Data["GNP"], y = mostPop30Data["Country"])
plt.xticks(rotation = 90)
plt.title("Top 30 Countries with the Highest GNP")
plt.xlabel("GNP")
plt.ylabel("Countries")
plt.show()

In [ ]:
df=df[df!=0].dropna()

#BAR GRAPH OF BOTTOM 30 COUNTRIES WITH LOWEST GNP
mostPop30Data = df.sort_values("GNP", ascending = True).head(30)

plt.figure(figsize = (10, 10))
sns.barplot(x = mostPop30Data["GNP"], y = mostPop30Data["Country"])
plt.xticks(rotation = 90)
plt.title("Bottom 30 Countries with the Lowest GNP")
plt.xlabel("GNP")
plt.ylabel("Countries")
plt.show()

In [ ]:
#BAR GRAPH OF TOP 30 COUNTRIES WITH HIGHEST POPULATIONS
mostPop30Data = country.sort_values("Population", ascending = False).head(30)

plt.figure(figsize = (10, 10))
sns.barplot(x = mostPop30Data["Population"], y = mostPop30Data["Country"])
plt.xticks(rotation = 90)
plt.title("Top 30 Countries with the Highest Populations")
plt.xlabel("Population (in billion)")
plt.ylabel("countries")
plt.show()

In [ ]:
#BAR GRAPH OF CONTINENTAL POPULATION
mostPopData = df.sort_values("Population", ascending = False)

plt.figure(figsize = (10, 10))
sns.barplot(x = mostPopData["Population"], y = mostPopData["Continent"])
plt.xticks(rotation = 90)
plt.title("Population per Continent")
plt.xlabel("Population")
plt.ylabel("Continent")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(data = df.nlargest(10, 'City_Population'), y = 'City', x = 'City_Population', palette='magma')
plt.title("TOP 10 Biggest Cities", size=16)
plt.xlabel(xlabel='Population', fontsize=14)
plt.show()

In [ ]:
#SIDE BAR GRAPH OF TOP 30 COUNTRIES WITH LARGEST AREA
largeArea30Data = country.sort_values("SurfaceArea", ascending = False).head(30)

plt.figure(figsize = (5, 10))
sns.barplot(x = largeArea30Data["SurfaceArea"], y = largeArea30Data["Country"])
plt.title("Top 30 Countries with the Largest Area")
plt.xlabel("Area")
plt.ylabel("Countries")
plt.show()

In [ ]:
#BAR GRAPH OF NUMBER OF COUNTRIES BY CONTINENT
continent = country.Continent.value_counts()
plt.figure(figsize=(10,7))
sns.barplot(x=continent.index,y=continent.values)
plt.xticks(rotation=45)
plt.ylabel('Number of countries')
plt.xlabel('Continent')
plt.title('Number of Countries by Continent',color = 'black',fontsize=20)

## <a name="p6">Visualization with Plotly</a>

In [ ]:
#Population per country
z = dict(type='choropleth',
            locations = df.Country,
            locationmode = 'country names', z = df.Population,
            text = df.Country, colorbar = {'title':'Population'},
            colorscale = 'Blackbody', reversescale = True)

layout = dict(title='Population per country',
geo = dict(showframe=False,projection={'type':'natural earth'}))
choromap = go.Figure(data = [z],layout = layout)
iplot(choromap,validate=False)

In [ ]:
#GNP per country
z = dict(type='choropleth',
locations = df.Country,
locationmode = 'country names', z = df.GNP,
text = df.Country, colorbar = {'title':'GNP per country'},
colorscale = 'Hot', reversescale = True)
layout = dict(title='GDP per Capita of World Countries',
geo = dict(showframe=False,projection={'type':'natural earth'}))
choromap = go.Figure(data = [z],layout = layout)
iplot(choromap,validate=False)

In [ ]:
#Area per country
z = dict(type='choropleth',
            locations = df.Country,
            locationmode = 'country names', z = df.SurfaceArea,
            text = df.Country, colorbar = {'title':'Total Area'},
            colorscale = 'Blackbody', reversescale = True)

layout = dict(title='Area per country',
geo = dict(showframe=False,projection={'type':'natural earth'}))
choromap = go.Figure(data = [z],layout = layout)
iplot(choromap,validate=False)

In [ ]:
#Life Expectancy per country
z = dict(type='choropleth',
            locations = df.Country,
            locationmode = 'country names', z = df.LifeExpectancy,
            text = df.Country, colorbar = {'title':'Life Expectancy'},
            colorscale = 'Blackbody', reversescale = True)

layout = dict(title='Life Expectancy per country',
geo = dict(showframe=False,projection={'type':'natural earth'}))
choromap = go.Figure(data = [z],layout = layout)
iplot(choromap,validate=False)

# Thank you so much, I hope you like it :)